<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_Pacman_Two_Agents_290420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [0]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Installation and Imports

Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.


In [3]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-liuqu2q7
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-liuqu2q7
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=e75c373dc00ca080a9191e1b876387c3870cb209d695ac5ff3ca2b10bcddd0ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-c6doja16/wheels/91/52/02/ec5c530fd76d56a66934ee91abbdae5240b766be1dc176deb7
Successfully built a2c-ppo-acktr
  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-tubmayad
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-tubmayad
     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=67cabc413cb7d86a28917389a5dd7c013abdc65382136903138b463c3bb738c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ic_gphzm/wheels/42/1c/91/28314e0c

In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/drive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
import datetime
import time

from benchmark.envs import *
from torch.autograd import Variable

import gym
from benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from custom.EarlyStopping import EarlyStopping_loss
from custom.model import *
from custom.custom_wrappers import DeterminsticNoopResetEnv 
# Imported required for the Model-based RL
#from sklearn.preprocessing import StandardScaler
from baselines.common.atari_wrappers import EpisodicLifeEnv

In [5]:
!wandb login ############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# General Functions

In [6]:
ENV_NAME = 'MsPacman-ramDeterministic-v4'
env = gym.make(ENV_NAME)
env = AtariARIWrapper(env)
obs = env.reset()
info_labels = env.labels()
print(info_labels)
key_list = ['enemy_blinky_x', 'enemy_blinky_y', 'player_x', 'player_y']
for key in info_labels:
    if key in key_list:
        print(key)

{'enemy_sue_x': 88, 'enemy_inky_x': 88, 'enemy_pinky_x': 88, 'enemy_blinky_x': 88, 'enemy_sue_y': 80, 'enemy_inky_y': 80, 'enemy_pinky_y': 80, 'enemy_blinky_y': 50, 'player_x': 88, 'player_y': 98, 'fruit_x': 0, 'fruit_y': 0, 'ghosts_count': 3, 'player_direction': 3, 'dots_eaten_count': 0, 'player_score': 0, 'num_lives': 2}
enemy_blinky_x
enemy_blinky_y
player_x
player_y


In [0]:
class General_functions():
    def __init__(self, ENV_NAME,n_actions,possible_positions):
        self.ENV_NAME = ENV_NAME
        self.env = gym.make(self.ENV_NAME)
        self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,66))        
        self.env = AtariARIWrapper(self.env)
        self.initial_info_labels = self.env.labels()
        self.key_list = ['enemy_blinky_x', 'enemy_blinky_y', 'player_x','player_y']
        
    
        self.prev_action_counter =  False
        self.repeated_end = False # Nawid -  Sets self.repeated end as false initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()

    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i-1] = 1
        return a

    def state_conversion(self,obs,info_labels):
        state = []          
        for key in info_labels:
            if key in self.key_list: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    
        state = np.array(state)
        return state
    
    def next_position(self,state, action):
        next_position = state[2:].copy()
        #print(next_position)
        if action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2
        
        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            return True
        else:
            return False
            
    ''' This is a general case which can be used with both the controller and the data collector
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action
            else:
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action
    '''

    
    def check_all_agents(self,info_label,next_info_label):
        repeated = np.equal(info_label,next_info_label).all()
        if repeated:
            self.repeated_end= True

    def check_state(self,state, next_state):
        #print(state[2:])
        repeated_state = np.equal(state[2:], next_state[2:]).all()
        if repeated_state:
            self.prev_action_counter = False


# Data collection and preprocessing

In [8]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME,n_actions, possible_positions):
        super(Data_collection,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.dataset_random = [] # Where the data is saved each time

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        '''
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        '''
        #print('D_train shape',D_train.shape)

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_act = np.array([act for _,_,_,_,act in D_train])
        
        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        #print('y train:',y_env_train[0:100,:])
        #print('X train:',X_train_obs[0:100,:])
        y_env_train = y_env_train.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        

        y_env_train = y_env_train - X_train_obs 
        
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.int16)
        y_env_val = y_env_val - X_val_obs 

        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        return env_train_data, env_val_data 

    def setup_dataset(self,env_train_data, env_val_data):
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
        return env_train_data, env_val_data
    
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action, None
            else:
                infeasible_action_one_hot = self.one_hot(action)
                #self.dataset_random.append([state, state, 0, False, infeasible_action_one_hot]) # NEED TO CHANGE WHERE THE DATA IS SAVED WHEN I AM USING THE DONES SINCE THE DONES ARE ADDED IN THE MAIN FUNCTION TO PAST VALUES   
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action, infeasible_action_one_hot
    
    def gather_random_trajectories(self,num_traj):
        #dataset_random = []
        
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)
            self.env = gym.make(self.ENV_NAME)
            self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,noop_max=66))
            self.env = AtariARIWrapper(self.env)
            obs = self.env.reset()
            
            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(obs, info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action
            
            

            while True:
                sampled_action, infeasible_action_one_hot = self.random_action_selection(state,prev_action)
                #sampled_action = np.random.randint(1,5)
                 
                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:
                    
                    if infeasible_action_one_hot is not None:
                        #print(infeasible_action_one_hot)
                        fake_next_state = np.zeros_like(state) #  Need to instantiate a new version each time to prevent updating a single variable which will affect all places(eg lists) where the variable is added
                        fake_next_state[0:2] = next_state[0:2].copy() # the enemy position of the fake next state is the current enemy position
                        fake_next_state[2:] = state[2:].copy() # The agent position for the fake next state is the state before any action was taken
                        #print('state:', state)
                        
                        #print('fake state', fake_next_state)
                        #print('diff', fake_next_state - state)
                        self.dataset_random.append([state, fake_next_state, 0, False, infeasible_action_one_hot])
                        #print('fake data added')
                        #print('fake state', fake_next_state)
                        #print('next state', next_state)


                    self.dataset_random.append([state, next_state, reward, done,sampled_action_one_hot])
                    #print('real data added')
                    #print('next state:', next_state)
                else:
                    self.dataset_random[-1][-2] = True # sets the previous value to done
                    done = True   
                #print('rand dataset:', self.dataset_random)
                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action
                if done:
                    break    
        #print(self.dataset_random[0:20])
        return self.dataset_random    
'''
ENV_NAME = 'MsPacman-ramDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.

possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
data_object = Data_collection(ENV_NAME,n_actions,possible_positions)
data1 = data_object.gather_random_trajectories(10)

rl_dataset = []
env_train, env_val = data_object.collate_data(data1,rl_dataset)
full_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)
'''

"\nENV_NAME = 'MsPacman-ramDeterministic-v4'\nn_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.\n\npossible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)\ndata_object = Data_collection(ENV_NAME,n_actions,possible_positions)\ndata1 = data_object.gather_random_trajectories(10)\n\nrl_dataset = []\nenv_train, env_val = data_object.collate_data(data1,rl_dataset)\nfull_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)\n"

In [9]:
x_train , y_train = full_env_train 
print(x_train[0:100,:])
y_train[0:100,:]

NameError: ignored

# Controller

In [0]:
class multi_model_based_control(General_functions):
    def __init__(self,ENV_NAME,n_actions,possible_positions,env_model,num_sequences,horizon_length):
        super(multi_model_based_control,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.env = gym.wrappers.Monitor(self.env, wandb.run.dir, video_callable=lambda episode_id: True if episode_id % 2 ==0 else False, force= True)
        self.env_model = env_model
        self.horizon_length = horizon_length
        self.num_sequences = num_sequences        
        self.n_actions = n_actions
        self.global_step = 0
        self.rl_dataset = []

    def random_sampling_shooting(self,real_obs):
        '''
        Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
        '''
        best_reward = -1e9
        best_next_action = []
        m_obs = np.array([real_obs for _ in range(self.num_sequences)])
        
        # array that contains the rewards for all the sequence
        unroll_rewards = np.zeros((self.num_sequences, 1)) 
        first_sampled_actions = []
        
        
        # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
        ## i.e roll a given number of trajectories in a single batch (to increase speed)
        for t in range(self.horizon_length):
            # sample actions for each sequence
            sampled_actions = np.array([np.random.randint(1,5) for _ in range(self.num_sequences)])
            sampled_actions_one_hot = np.array([self.one_hot(action) for action in sampled_actions])
        
            env_model_input = np.concatenate([m_obs, sampled_actions_one_hot], axis = 1)            
            # compute the next state for each sequence
            pred_obs = self.env_model(torch.tensor(env_model_input).to(self.device))
            print('m obs:',m_obs)
            print('sampled_action:',sampled_actions)
            print('pred_obs:', pred_obs)


            # add previous observation
            next_obs = pred_obs.cpu().detach().numpy() + m_obs
            #print('next observation',next_obs)

            #print('player',m_obs[:,8:10])
        
            m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards

            if t ==0:
                first_sampled_actions = sampled_actions 


        # Best the position of the sequence with the higher reward
        
        random_choice = np.random.randint(0,self.num_sequences)

        # take the first action of this sequence
        best_action = first_sampled_actions[random_choice]
        #print('best action',best_action)
        #best_action =  np.squeeze(best_action)
        return best_action
    
    def on_policy_collection(self,STEPS_PER_AGGR):
        obs = self.env.reset()
        self.env.seed(0) # Set the random seed of the environment
        num_examples_added = 0

        # records how long the agent survives for
        timesteps = 0
        wait_done = False # Use to wait until the episode ends after the agent has lot all of its lives and has been eaten
        # records how many simulations have occurred
        
        info_labels = self.env.labels()
        state = self.state_conversion(obs,info_labels)
        next_state = state.copy() # Initially the state and the next state will be the same
        while num_examples_added < STEPS_PER_AGGR:
            while True:
                                                
                state = self.state_conversion(obs,info_labels)
                action = self.random_sampling_shooting(state)
                    
                action_one_hot = self.one_hot(action)
                # one step in the environment with the action returned by
                next_obs, reward, done, next_info = self.env.step(action)
                next_info_labels = next_info['labels']                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.rl_dataset.append([state, next_state, reward, done, action_one_hot])
                                
                obs = next_obs
                info_labels = next_info_labels
                num_examples_added += 1 

                if done:
                    obs = self.env.reset()
                    info_labels = self.env.labels()
                    self.env.seed(0)
                    break
        return self.rl_dataset  

# Hyperparameters

In [10]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")#,monitor_gym=True)
ENV_NAME = 'MsPacman-ramDeterministic-v4' #'MsPacmanNoFrameskip-v4'  #'MsPacmanDeterministic-v4' # 'MsPacmanNoFrameskip-v4'
 
# Main loop hyperp
env_model_pretrain = False
pretrained_env_model = '/content/drive/My Drive/MsPacman-data/Env model/Env_model_21_4_2020-8_37.pt'#'/content/gdrive/My Drive/MsPacman-data/Env model/Env_model_17_4_2020-11_0.pt' #'/content/Env_model_17_4_2020-10_32.pt' #'/content/Epoch_no_43_Env_model_13_4_2020-8_22_checkpoint.pt'
ENV_LEARNING_RATE = 1e-4

AGGR_ITER = 10
STEPS_PER_AGGR = 1000

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 6000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Supervised Model Hyperp
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  10000

# Controller Hyperp
HORIZON_LENGTH = 1
NUM_ACTIONS_SEQUENCES = 1

load_data = True
if load_data:
    loaded_trajectories = '/content/drive/My Drive/MsPacman-data/rand_traj_6000-30_4_2020-14_16.npy' #'/content/drive/My Drive/MsPacman-data/rand_traj_3000-30_4_2020-11_27.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj_allowed_move_prev_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-28_4_2020-18_0.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_3000_traj_rand+allowed_moves.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-21_4_2020-7_47.npy'#'/content/gdrive/My Drive/MsPacman-data/Random trajectories/No_wall_movements/rand_traj_3000-_all_lives_16_4_2020-14_33.npy'#'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-16_4_2020-14_33.npy'#

collect_data = False

observation_channels = 1
action_dim = 1
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)

possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0

if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)

cuda


In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR

config.train_model_iter = TRAIN_ITER_MODEL
config.env_lr = ENV_LEARNING_RATE
config.env_model_pretrain = env_model_pretrain
config.pretrained_env =  pretrained_env_model

config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.random_seed = random_seed

if load_data:
    config.loaded_trajectories = loaded_trajectories

# Main - Model training

Data collection and processing

In [12]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,n_actions,possible_positions)
if load_data:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
    filename = '/content/drive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
    np.save(filename,rand_dataset)

#rand_dataset = np.array([x for x in rand_dataset if not same(x)])
#rand_dataset = rand_dataset[0:100,:]
empty_dataset = []
env_train_data, env_val_data = data_collector.collate_data(rand_dataset,empty_dataset)

norm_env_train_data, norm_env_val_data = data_collector.setup_dataset(env_train_data, env_val_data)

number random examples: 867321 len(D_train_rand) 693857 len(D_valid_rand) 173464


Dynamics model training

In [13]:
env_model = NNDynamicsModel(n_actions + 4, 4).to(device) # Nawid - Need to put both actions as it is a one-hot vector

if env_model_pretrain:
    print('checkpoint')
    #env_model.load_state_dict(torch.load(pretrained_env_model))
    checkpoint = torch.load(pretrained_env_model)
    env_model.load_state_dict(checkpoint['model_state_dict'])
    
    for parameter in env_model.parameters():
        parameter.requires_grad = False
    env_model.eval()

else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=env_model_name)
    for n_iter in range(AGGR_ITER):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break 
        train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

#rew_model = Reward_Oracle()

  0%|          | 1/10000 [00:18<51:32:54, 18.56s/it]

Validation loss decreased/improved for Env_model_30_4_2020-15_33  (100000000000.000000 --> 1.616476).  Saving model ...
epoch: 0 completed


  0%|          | 2/10000 [00:37<51:48:42, 18.66s/it]

Validation loss decreased/improved for Env_model_30_4_2020-15_33  (1.616476 --> 1.206917).  Saving model ...
epoch: 1 completed


  0%|          | 3/10000 [00:56<52:02:37, 18.74s/it]

...

EarlyStopping for Env_model_30_4_2020-15_33 counter: 2 out of 5
epoch: 27 completed


  0%|          | 29/10000 [09:14<52:37:23, 19.00s/it]

EarlyStopping for Env_model_30_4_2020-15_33 counter: 3 out of 5
epoch: 28 completed


  0%|          | 30/10000 [09:33<52:49:17, 19.07s/it]

EarlyStopping for Env_model_30_4_2020-15_33 counter: 4 out of 5
epoch: 29 completed
EarlyStopping for Env_model_30_4_2020-15_33 counter: 5 out of 5
Env_model_30_4_2020-15_33 has stopped
Early stopping
Early stopping


Training the reward function and MPC

In [14]:

#rand_env_model = NNDynamicsModel(n_actions + 4, 4).to(device) # Nawid - Need to put both actions as it is a one-hot vector
#rand_env_model.eval()
env_model.eval()
Controller = multi_model_based_control(ENV_NAME,n_actions,possible_positions, env_model, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH)
for n_iter in range(AGGR_ITER):    
    '''
    if early_stopping_env.early_stop:~
        print('Early stopping')
        break 
    '''
    #train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)    
    rl_dataset = Controller.on_policy_collection(100000)

Controller.env.close()
    
'''
    if collect_data:
        env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)
        norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data = data_collector.setup_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
'''

Streaming output truncated to the last 5000 lines.
sampled_action: [2]
pred_obs: tensor([[-0.5453,  0.7064, -0.1542,  0.7959]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[114 134 134 144]]
sampled_action: [4]
pred_obs: tensor([[-0.7721,  0.7126,  0.1418,  4.2621]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[114 134 134 144]]
sampled_action: [4]
pred_obs: tensor([[-0.7721,  0.7126,  0.1418,  4.2621]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[114 134 134 144]]
sampled_action: [2]
pred_obs: tensor([[-0.5453,  0.7064, -0.1542,  0.7959]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[114 134 134 144]]
sampled_action: [3]
pred_obs: tensor([[-0.6342,  0.8103,  0.2605,  0.3291]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[88 50 88 98]]
sampled_action: [1]
pred_obs: tensor([[-2.6186e+00,  8.2413e-04,  1.0267e-01,  3.5052e-01]], device='cuda:0',
       grad_fn=<AddmmBackward>)
m obs: [[88 50 88 98]]
sampled_action: [4]
pre

KeyboardInterrupt: ignored

# Old Code

In [0]:
loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy'
loaded_dataset = np.load(loaded_trajectories,allow_pickle=True)

loaded_trajectory_2 = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_50.npy'
loaded_dataset2 = np.load(loaded_trajectory_2,allow_pickle=True)

collated_dataset = np.concatenate((loaded_dataset,loaded_dataset2),axis=0)
filename = '/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix'
np.save(filename,collated_dataset)